In [ ]:
import os
import cv2
import numpy as np
import torch
import dlib
from imutils import face_utils
from basicsr.archs.rrdbnet_arch import RRDBNet
from basicsr.utils.download_util import load_file_from_url
from torchvision import transforms  
import torch.nn as nn
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.nn.functional as F
import math
import time
from imutils import face_utils
from scipy.spatial import distance as dist
import matplotlib.pyplot as plt
import mediapipe

import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sys.path.append("../../")
from utils import *
from BaseModels.resnetModels import BB_model
from BaseModels.efficientnetModels import BB_model
# device = "cpu"


    


In [76]:

    
# Latest Model
# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Segmentation/Regression_model_1.140251636505127.pth"

# Mix Dataset
# regression_model_path = '/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/Regression_model_1.487574208665777.pth'

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModels/SpheroPipeLine/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/Trained_Models/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "RecentModels/BioGi4eH2head/Regression_model_0.7620949026704394.pth"

# regression_model_path = "RecentModels/SpheroPipeLine/Resnet_32_IMG_SIZE__32/Regression_model_0.25639680131442016.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/RecentModels/AllDataset/Regression_model_1.4074174204430023.pth"

# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/Recent_EffcientNetModels/EfficientNet1_6_smoothl1loss_batch_64.pt"


# regression_model_path = "/home/nipun/Documents/Uni_Malta/LuminEye/LuminEye-Iris-Center-Localization/RecentModelsonGpu/Recent_EffcientNetModels/EfficientNet1_6_smoothl1loss_batch_256.pt"


# regression_model_path = "RecentModelsonGpu/Recent_EffcientNetModels/Regression_EffcientNet__epoch_500_SmoothL1Loss_summation_batch_256_resize_64_for_gi4e_bioid_h2head_1.52.pt"

# regression_model_path = "RecentModelsonGpu/Recent_EffcientNetModels/EfficientNetWingLossV1.pt"

# regression_model_path = 'ResnetBaseWithMSEMEANwithoughtCoordConv/checkpoint.pt'


regression_model_path = 'EffcienientNetNOTCoordconv.pt'

REGRESSION_MODEL = load_model(
    model_path=regression_model_path)



RESIZE_AMT = 64



In [77]:
# BioID
val_csv_path = "/home/nipun/Documents/Uni_Malta/Datasets/CenterRegression/MP2GAZE/AllCoordinatesMp2GazeTest.csv"


val_df = pd.read_csv(val_csv_path)



In [78]:
val_df.head()

,ImageName,Coordinates,DataSet Type
0,"MPIIGaze/Data/Original/p00/day02/0350.jpg""","[576.75131084363, 446.9430824441167, 595.62107...",MP2GAZE
1,"MPIIGaze/Data/Original/p00/day23/0201.jpg""","[629.1027244977723, 418.5422721626321, 649.159...",MP2GAZE
2,"MPIIGaze/Data/Original/p00/day22/0437.jpg""","[519.3837103984533, 379.3424773235636, 551.963...",MP2GAZE
3,"MPIIGaze/Data/Original/p00/day34/0296.jpg""","[563.4499942709884, 374.6517620194357, 591.877...",MP2GAZE
4,"MPIIGaze/Data/Original/p00/day10/0198.jpg""","[659.4630047049818, 463.58724173032533, 685.19...",MP2GAZE


In [82]:
def cropeyes(image, inner_array, outer_array):

    arr = {"top_left": [inner_array[0]-5, inner_array[1]-20],
           "bottom_right": [outer_array[0]+5, outer_array[1]+20]}

    return image[arr["top_left"][1]:arr["bottom_right"][1], arr["top_left"][0]:arr["bottom_right"][0]], arr

In [83]:



def rescale_coordinate(coord,original_image,resize_amt):
    
    h,w = original_image.shape[:2]
    coord[0] = int((coord[0]/resize_amt) * w)
    coord[1] = int((coord[1]/resize_amt) * h)
    
    return coord

In [84]:
def scaleCoorinatesToOriginalImage(pred_coords,eye_margin):
    
    # {'top_left': array([385, 214]), 'bottom_right': array([426, 226])}
    
    x1 = eye_margin["top_left"][0] + pred_coords[0]
    y1 = eye_margin["top_left"][1] + pred_coords[1]
    
    return [x1,y1]
    

In [85]:
def calculateEuclideanDistance(coord1,coord2):
    return (((coord1[0])-float(coord2[0]))^2  + (float(coord1[0])-float(coord2[0]))^2) ^ 0.5

In [86]:
IMG_DIR = "/home/nipun/Documents/Uni_Malta/Datasets/"

In [ ]:
maximizedNormalizedError = []
for i, (_, rows) in enumerate(val_df.iterrows()):

        bbox = [int(float(x)) for x in rows["Coordinates"][1:-1].split(',')]

        left_inner = bbox[0:2]
        left_center = bbox[2:4]
        left_outer = bbox[4:6]

        right_inner = bbox[6:8]
        right_center = bbox[8:10]
        right_outer = bbox[10:12]

        img_path = os.path.join(IMG_DIR, rows["ImageName"][:-1])
        # #
        img = cv2.imread(img_path)


        IPD = distance.euclidean(left_center, right_center)
        

       

        left_eye, Leye = cropeyes(img, left_inner, left_outer)
        right_eye, Reye = cropeyes(img, right_inner, right_outer)
        
        
        _, pred_l_eye = prediction_imageV2(
            model=REGRESSION_MODEL, image=left_eye,resize_amt=RESIZE_AMT)

        _, pred_r_eye = prediction_imageV2(
            model=REGRESSION_MODEL, image=right_eye,resize_amt=RESIZE_AMT)
        
        
        pred_l_eye = rescale_coordinate(pred_l_eye, left_eye, RESIZE_AMT)

        pred_r_eye = rescale_coordinate(pred_r_eye, right_eye, RESIZE_AMT)
        
        
        pred_l_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            pred_l_eye, Leye)
        pred_r_eye_toOriginaImage = scaleCoorinatesToOriginalImage(
            pred_r_eye, Reye)
        
        
        cv2.circle(img, (int(pred_l_eye_toOriginaImage[0]), int(
            pred_l_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)

        cv2.circle(img, (int(pred_r_eye_toOriginaImage[0]), int(
            pred_r_eye_toOriginaImage[1])), 1, (0, 255, 0), -1)
        
        
        cv2.circle(img, (int(left_inner[0]), int(
            left_inner[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(left_center[0]), int(
            left_center[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(left_outer[0]), int(
            left_outer[1])), 1, (0, 0, 255), -1)

        cv2.circle(img, (int(right_inner[0]), int(
            right_inner[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(right_center[0]), int(
            right_center[1])), 1, (0, 0, 255), -1)
        cv2.circle(img, (int(right_outer[0]), int(
            right_outer[1])), 1, (0, 0, 255), -1)
        
         
          
        left_eye_euclidea_distance = distance.euclidean(
            pred_l_eye_toOriginaImage, left_center)

        right_eye_euclidea_distance = distance.euclidean(
            pred_r_eye_toOriginaImage, right_center)
        
        eMax = max(left_eye_euclidea_distance, right_eye_euclidea_distance)/IPD

        print(eMax)

        maximizedNormalizedError.append(eMax)
    
        # break

0.02255384108019632
0.016765219143761074
0.0
0.026126497213658204
0.03217643201321576
0.01563339910144465
0.01248362597931077
0.01288890976575717
0.008163605321754816
0.026572823627058487
0.02046953170891141
0.02049584873004486
0.006112464870141717
0.01945768347613674
0.015514787986130315
0.020461495722482567
0.03243574962539294
0.020643253864456434
0.014517167628488535
0.02388410885962159
0.027700701129007948
0.011145218132094787
0.01642569885854426
0.014063042481340066
0.023596458909150436
0.01817280910462035
0.028447458770443367
0.032584421552822575
0.02343312028655591
0.01552787542996996
0.011974115989261225
0.007574888148133693
0.05233820578865744
0.007983666160462774
0.015540573797716228
0.01208244186660354
0.011896334946882258
0.007467574778976377
0.028239787090084047
0.01492371097349751
0.029411255865026874
0.018773922658852197
0.015037168948034583
0.019611613513818404
0.010959932487023821
0.011497671238805652
0.01552824984374854
0.018469811292500967
0.01049785603183589
0.00698

In [89]:
e_0_25 = CheckForLess(maximizedNormalizedError,0.25)

e_0_05 = CheckForLess(maximizedNormalizedError,0.05)

e_0_1 = CheckForLess(maximizedNormalizedError,0.1)

In [90]:
(len(e_0_25)/len(maximizedNormalizedError)) *100

100.0

In [91]:
(len(e_0_05)/len(maximizedNormalizedError)) * 100

94.53125

In [92]:
len(e_0_1)/len((maximizedNormalizedError)) * 100

100.0

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(img[:,:,::-1])
